In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier, DecisionTreeClassifier
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer

import nltk.classify
import sklearn.svm
from sklearn.svm import LinearSVC


In [2]:
def read_train():
    train=pd.read_csv("./tweet-sentiment-extraction/train.csv")
    train['text']=train['text'].astype(str)
    train['selected_text']=train['selected_text'].astype(str)
    return train

def read_test():
    test=pd.read_csv("./tweet-sentiment-extraction/test.csv")
    test['text']=test['text'].astype(str)
    return test

def read_submission():
    test=pd.read_csv("./tweet-sentiment-extraction/sample_submission.csv")
    return test
    
train_df = read_train()
test_df = read_test()
submission_df = read_submission()

In [3]:
import re, string, random

def remove_noise(tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [4]:
#train_df[['text', 'sentiment']]
positive_tweets = []
negative_tweets = []
neutral_tweets = []

positive_test_tweets = []
negative_test_tweets = []
neutral_test_tweets = []


def extract_texts(df, sentiment, tweets_table):
    for index, row in df.iterrows():
        if(row['sentiment'] == sentiment):
            tweets_table.append(row['text'])

        
extract_texts(train_df, 'positive', positive_tweets)
extract_texts(train_df, 'negative', negative_tweets)
extract_texts(train_df, 'neutral', neutral_tweets)

extract_texts(test_df, 'positive', positive_test_tweets)
extract_texts(test_df, 'negative', negative_test_tweets)
extract_texts(test_df, 'neutral', neutral_test_tweets)


#!!!!!!!!!!

#positive_tweets = positive_tweets + positive_test_tweets
#negative_tweets = negative_tweets + negative_test_tweets
#neutral_tweets = neutral_tweets + neutral_test_tweets

#!!!!!!!!!
        
    

In [5]:
print(len(neutral_tweets))


11118


In [6]:

    tknzr = TweetTokenizer()
    stop_words = stopwords.words('english')
    
    def tokenize_array(array, result):
        for f in array:
            result.append(tknzr.tokenize(f))
    

    positive_tweet_tokens = []
    negative_tweet_tokens = []
    neutral_tweet_tokens = []
    
    positive_test_tweet_tokens = []
    negative_test_tweet_tokens = []
    neutral_test_tweet_tokens = []
    
    tokenize_array(positive_tweets, positive_tweet_tokens)
    tokenize_array(negative_tweets, negative_tweet_tokens)
    tokenize_array(neutral_tweets, neutral_tweet_tokens)
    
    tokenize_array(positive_test_tweets, positive_test_tweet_tokens)
    tokenize_array(negative_test_tweets, negative_test_tweet_tokens)
    tokenize_array(neutral_test_tweets, neutral_test_tweet_tokens)

In [7]:
    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []
    neutral_cleaned_tokens_list = []
    
    positive_cleaned_test_tokens_list = []
    negative_cleaned_test_tokens_list = []
    neutral_cleaned_test_tokens_list = []


    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
        
    for tokens in neutral_tweet_tokens:
        neutral_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
        
        
    for tokens in positive_test_tweet_tokens:
        positive_cleaned_test_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_test_tweet_tokens:
        negative_cleaned_test_tokens_list.append(remove_noise(tokens, stop_words))
        
    for tokens in neutral_test_tweet_tokens:
        neutral_cleaned_test_tokens_list.append(remove_noise(tokens, stop_words))

    pos_words = get_all_words(positive_cleaned_tokens_list)
    neg_words = get_all_words(negative_cleaned_tokens_list)
    net_words = get_all_words(neutral_cleaned_tokens_list)
    
    pos_test_words = get_all_words(positive_cleaned_test_tokens_list)
    neg_test_words = get_all_words(negative_cleaned_test_tokens_list)
    net_test_words = get_all_words(neutral_cleaned_test_tokens_list)


    freq_dist_pos = FreqDist(pos_words)
    print(freq_dist_pos.most_common(10))

[('...', 1370), ('day', 1345), ('good', 1284), ('love', 1012), ('happy', 865), ('get', 739), ('go', 636), ('thanks', 559), ('mother', 533), ('..', 508)]


In [8]:
    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
    neutral_tokens_for_model = get_tweets_for_model(neutral_cleaned_tokens_list)

    positive_tokens_for_test = get_tweets_for_model(positive_cleaned_test_tokens_list)
    negative_tokens_for_test = get_tweets_for_model(negative_cleaned_test_tokens_list)
    neutral_tokens_for_test = get_tweets_for_model(neutral_cleaned_test_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]
    
    neutral_dataset = [(tweet_dict, "Neutral")
                         for tweet_dict in neutral_tokens_for_model]

    dataset = positive_dataset + negative_dataset + neutral_dataset#!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    
    
    positive_test_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_test]

    negative_test_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_test]
    
    neutral_test_dataset = [(tweet_dict, "Neutral")
                         for tweet_dict in neutral_tokens_for_test]

    train_data = positive_dataset + negative_dataset + neutral_dataset
    
    test_data = positive_test_dataset + negative_test_dataset + neutral_test_dataset
    

In [9]:
print(len(train_data))

27481


In [10]:
    from nltk.classify.scikitlearn import SklearnClassifier


    classifierNaiveBayes = NaiveBayesClassifier.train(train_data)
    
    #classifierDecisionTree = DecisionTreeClassifier.train(train_data)

    classifierLinearSVC = SklearnClassifier(LinearSVC()).train(train_data)



    print("Accuracy classifierNaiveBayes is:", classify.accuracy(classifierNaiveBayes, test_data))
    
    #print("Accuracy classifierDecisionTree is:", classify.accuracy(classifierDecisionTree, test_data))

    print("Accuracy classifierLinearSVC is:", classify.accuracy(classifierLinearSVC, test_data))


Accuracy classifierNaiveBayes is: 0.6445953593661573
Accuracy classifierLinearSVC is: 0.677985285795133


In [11]:
#from nltk.classify.scikitlearn import SklearnClassifier
#from sklearn.naive_bayes import MultinomialNB,BernoulliNB


#MNB_classifier = SklearnClassifier(MultinomialNB())
#MNB_classifier.train(train_data)
#print("MultinomialNB accuracy percent:",nltk.classify.accuracy(MNB_classifier, test_data))

#BNB_classifier = SklearnClassifier(BernoulliNB())
#BNB_classifier.train(train_data)
#print("BernoulliNB accuracy percent:",nltk.classify.accuracy(BNB_classifier, test_data))

In [12]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

print("Naive Bayes Algorithm accuracy percent:", (nltk.classify.accuracy(classifierNaiveBayes, test_data))*100)
classifierNaiveBayes.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(train_data)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, test_data))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(train_data)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, test_data))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(train_data)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, test_data))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(train_data)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, test_data))*100)

SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(train_data)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, test_data))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(train_data)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, test_data))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(train_data)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, test_data))*100)

Naive Bayes Algorithm accuracy percent: 64.45953593661574
Most Informative Features
                     sad = True           Negati : Positi =     66.6 : 1.0
                 welcome = True           Positi : Negati =     60.7 : 1.0
                   thank = True           Positi : Negati =     50.1 : 1.0
                 mothers = True           Positi : Negati =     47.0 : 1.0
                    bore = True           Negati : Positi =     46.1 : 1.0
               wonderful = True           Positi : Negati =     43.8 : 1.0
               hopefully = True           Positi : Negati =     40.8 : 1.0
                    hate = True           Negati : Positi =     38.5 : 1.0
                 awesome = True           Positi : Negati =     38.3 : 1.0
                    wars = True           Positi : Negati =     35.4 : 1.0
                   sadly = True           Negati : Positi =     34.2 : 1.0
                    suck = True           Negati : Positi =     33.2 : 1.0
                

C:\APPS\ANACONDA\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression_classifier accuracy percent: 70.17543859649122
SGDClassifier_classifier accuracy percent: 71.30730050933785
SVC_classifier accuracy percent: 70.6847764572722
LinearSVC_classifier accuracy percent: 67.79852857951329
NuSVC_classifier accuracy percent: 71.44878324844369
